In [2]:
%load_ext autoreload
%autoreload 2

This notebook demonstrates synthetic data generation using LLMs (GPT-4, DeepSeek, Qwen) via the DSPy library. The process involves:

1. Selecting relevant bootstrap examples
2. Using LLMs to generate additional data
3. Filtering out irrelevant examples
4. Using final curated data for model training

# 1. Data preparation 

In [3]:
import dotenv
# Load environment variables, openai api key
dotenv.load_dotenv()

import dspy
from speedy_utils.all import *
from eedi.data.common import *

lm = dspy.LM("gpt-4o-mini")

2025-01-06 01:05:30.716 | INFO     | eedi.data.common:<module>:387 - common: ['VectorDBRetriever', 'prepare_db_train_val', 'preproc_df', 'get_df_parsed', 'format_df_generic', 'compute_metric', 'mine_hard_negatives', 'mine_hard_negatives', 'extract_tag', 'TEMPLATE_INPUT_V3', 'df_mapping']


### 1.1 Generate synthetic data

For implementation details, see:
- `data/retriver/01_generate_plan.ipynb`
- `data/retriver/02_generate_synthetic_data.ipynb`

Note: Generated data is provided in the `data` folder for direct use.

### 1.2 Load and preprocess training data

The training data is composed of the competition's provided data and the generated synthetic data.

In [4]:
# df_train_val

In [5]:
df_train_val = pd.read_csv("./data/train.csv")
# split fold
df_train = df_train_val[df_train_val.QuestionId % 5 != 0].copy()
df_val = df_train_val[df_train_val.QuestionId % 5 == 0].copy()

df_miscon = pd.read_csv("./data/misconception_mapping.csv")
df_train_flat = preproc_df(df_train, df_miscon, is_train=True)
df_train_flat = df_train_flat.dropna(subset=["MisconceptionName"])
df_val_flat = preproc_df(df_val, df_miscon, is_train=True)

### 1.2.1 Data curation
The synthedic data might contain low-quality examples. We will filter out these examples using a prompt-based filtering approach.
- Filter low-quality examples.
    - Framework: dspy
    - Method: prompt-based filtering

In [6]:
from textwrap import dedent

class DataCuratorSignature(dspy.Signature):
    """Analyze how well an incorrect answer reflects a suspected misconception in a mathematics problem.
    The goal is to determine whether there is a clear, logical connection between the misconception and the wrong answer.

    You task:
    - Analyze the problem data and the suspected misconception
       - Explain misconception->wrong answer connection
       - Score (0-10):
         10: Perfect alignment
         8-9: Strong alignment
         5-7: Moderate alignment
         1-4: Weak alignment
         0: No alignment

    Guidelines:
    - Focus on misconception-answer connection
    - Be specific about error path
    - Flag assumptions
    - Consider consistency
    """

    problem_data = dspy.InputField(description="Problem data")
    scratchpad: str = dspy.OutputField(
        description="Analysis of the problem and misconception"
    )
    evaluation_score: int = dspy.OutputField(
        description="Evaluation explanation and score (0-10)"
    )


def format_prompt_for_curate(query, pos):
    # use query and pos[0] to fill in the template
    PROBLEM_DATA = "{}\nSuspected misconception: {}".format(query, pos)
    return PROBLEM_DATA

curator_program = dspy.Predict(DataCuratorSignature)

In [7]:
# Load synthetic data and format prompts

df_synthetic = pd.read_parquet("./data/synthetic_data.parquet")
df_synthetic["PROMPT"] = df_synthetic.apply(
    lambda x: TEMPLATE_INPUT_V3.format(**x), axis=1
)

def run_curator_on_synthetic_data(row):
    prompt = format_prompt_for_curate(row["PROMPT"], row["MISCONCEPTION_NAME"])
    output = curator_program(problem_data=prompt, lm=lm)
    return output.evaluation_score

df_synthetic["EVALUATION_SCORE"] = multi_thread(run_curator_on_synthetic_data, df_synthetic, workers=128)

# Keep only high-quality synthetic data
print('Drop low quality data:', (df_synthetic["EVALUATION_SCORE"] < 8).sum())

df_synthetic = df_synthetic[df_synthetic["EVALUATION_SCORE"] >= 8]

Multi-thread, Function: `run_c..: 100%|██████████████████████████████| 2587/2587 [00:06<00:00, 413.69it/s, SUCCESS=2587]

Multi thread results:
+---------+---------+
| Key     |   Value |
+=========+=========+
| SUCCESS |    2587 |
+---------+---------+
Drop low quality data: 332


##### DEBUG

In [8]:
lm.inspect_history(1)





[2025-01-06T01:05:41.159336]

System message:

Your input fields are:
1. `problem_data` (str): Problem data

Your output fields are:
1. `scratchpad` (str): Analysis of the problem and misconception
2. `evaluation_score` (int): Evaluation explanation and score (0-10)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## problem_data ## ]]
{problem_data}

[[ ## scratchpad ## ]]
{scratchpad}

[[ ## evaluation_score ## ]]
{evaluation_score}        # note: the value you produce must be a single int value

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Analyze how well an incorrect answer reflects a suspected misconception in a mathematics problem.
        The goal is to determine whether there is a clear, logical connection between the misconception and the wrong answer.
        
        You task:
        - Analyze the problem data and the suspected misconception
           - Explain misconception->wro

Print some examples to check the quality of the synthetic data.

In [9]:
from IPython.display import display

print("Train data examples:", "TOTAL:", len(df_train_flat))
display(df_train_flat.head(3)[["Prompt", "MisconceptionName"]])
print("Synthetic data examples:", "TOTAL:", len(df_synthetic))

display(df_synthetic.head(3)[["PROMPT", "MISCONCEPTION_NAME"]])

Train data examples: TOTAL: 3498


,Prompt,MisconceptionName
0,"- **Question**: Simplify the following, if pos...",Does not know that to factorise a quadratic ex...
1,"- **Question**: Simplify the following, if pos...",Thinks that when you cancel identical terms fr...
2,"- **Question**: Simplify the following, if pos...",Does not know that to factorise a quadratic ex...


Synthetic data examples: TOTAL: 2255


,PROMPT,MISCONCEPTION_NAME
0,"Question: In a triangle, the measures of the a...",Does not know that angles in a triangle sum to...
1,Question: Calculate:\n\(\n\frac{2}{3} \times \...,Uses dividing fractions method for multiplying...
2,Question: What is the sum of all angles around...,Believes there are 100 degrees in a full turn


### Finalize training data
The final training data is a combination of the original training data and the high-quality synthetic data.
Each item is come with query and pos (misconception name) for the data curator to analyze.

In [10]:
items = []
queries = df_train_flat["Prompt"].tolist() + df_synthetic["PROMPT"].tolist()
misconceptions = (
    df_train_flat["MisconceptionName"].tolist()
    + df_synthetic["MISCONCEPTION_NAME"].tolist()
)
for query, misconception in zip(queries, misconceptions):
    item = {
        "query": query,
        "pos": [misconception],
    }
    items.append(item)
print("Saving data for training the retriever...", 'total:', len(items))

dump_json_or_pickle(items, "./data/retriver/train_items.jsonl")

Saving data for training the retriever... total: 5753


In [11]:
# Also save the candidate pool since the avaialble positive only cover 2/3 of the misconceptions
train_misconceptions = df_mapping['MisconceptionName'].tolist()
train_misconceptions = [{'text': x} for x in train_misconceptions if x]
dump_json_or_pickle(train_misconceptions, '/tmp/candidate_pool.jsonl')
print('Num misconceptions:', len(train_misconceptions))

Num misconceptions: 2587


# 2. Model Finetuning



The model finetuning process involves:

1. Tokenization and length filtering using FLAGS embedding framework
2. Training configuration:
    - Learning rate: 1e-5
    - Batch size: 32 
    - Max sequence length: 128
    - Warmup steps: 500
    - Weight decay: 0.01
    - Training epochs: 3
3. Adding special tokens for retrieval optimization:
    - Special token `<response>` added to sequence end
    - Last token embedding used for similarity computation
4. Training process:
    - Compute cosine similarity between query and misconception embeddings
    - Optimize embeddings through contrastive learning
    - Early stopping based on validation performance
    - Gradient clipping for stable training
    
Implementation note: Qwen models require special handling since they were not pre-trained for retrieval tasks. We add an extra token at sequence end and use its embedding for similarity matching during both training and inference.

### Iterative Hard Negative Mining Process

The training process involes iterative hard negative mining to improve the model's performance. The process is as follows:

| **Round** | **Stage**       | **Command** |
|-----------|-----------------|------------|
| **1**     | Hard Negative Mining | `scripts/mine_hn.sh BAAI/bge-base-en-v1.5  data/retriver/train_items.jsonl data/retriver/hn_r1.jsonl` |
| **1**     | Training        | `scripts/train.sh data/retriver/hn_r1.jsonl outputs/models/0.5B/round1/` |
| **2**     | Hard Negative Mining | `scripts/mine_hn.sh outputs/models/0.5B/round1/ data/retriver/train_items.jsonl data/retriver/hn_r2.jsonl` |
| **2**     | Training        | `scripts/train.sh data/retriver/hn_r2.jsonl outputs/models/0.5B/round2/` |
| **3**     | Hard Negative Mining | `scripts/mine_hn.sh outputs/models/0.5B/round2/ data/retriver/train_items.jsonl data/retriver/hn_r3.jsonl` |
| **3**     | Training        | `scripts/train.sh data/retriver/hn_r3.jsonl outputs/models/0.5B/round3/` |


### Notes:
- **Hard Negative Mining:** Uses the previous round's trained model to mine hard negatives for better data.
- **Training Parameters:** Learning rate, batch size, and other configurations are consistent across rounds.
- **Iterations:** This process refines the model iteratively to improve its retrieval performance.

# 3. Evaluation 

### 3.1 Model Service (app/embed_service_app.py)

The Flask application serves the trained model as an embedding service with the following features:

1. **Model Loading**
    - Loads quantized model from checkpoint
    - Configures model parameters and special tokens
    - Handles both base model and LoRA weights

2. **REST API**
    - Endpoint: `/v1/embeddings`
    - Method: POST
    - Input: JSON with text array
    - Output: Embedding vectors 

3. **Request Processing**
    - Batches input text
    - Applies tokenization and padding
    - Returns embeddings in JSON format

4. **Error Handling**
    - Input validation
    - Exception catching
    - Informative error messages

5. **Resource Management** 
    - Efficient batch processing
    - GPU memory optimization
    - Request queueing
```

In [13]:
# !python app/embed_service_app.py --base_model Qwen/Qwen2.5-0.5B --lora_path ./outputs/models/0.5B/round1/ 

In [14]:
import requests

def get_embeddings(texts, url = "http://0.0.0.0:8080/v1/embeddings"):
     """
     Get embeddings for a list of texts from the local embedding service
     
     Args:
          texts (list): List of strings to get embeddings for
     
     Returns:
          dict: Response from the embedding service
     """
     
     payload = {"texts": texts}
     response = requests.post(url, json=payload)
     response =  response.json()
     return np.array(response["embeddings"])

In [15]:
df_val_flat.head(1)

,QuestionId,ConstructName,SubjectName,CorrectAnswer,QuestionText,CorrectAnswerText,Option,AnswerText,MisconceptionId,MisconceptionName,Prompt
0,0,Use the order of operations to carry out calcu...,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),B,\( 3 \times 2+(4-5) \),NaN,None,- **Question**: \[\n3 \times 2+4-5\n\]\nWhere ...


In [17]:
from transformers import AutoTokenizer
import json


def get_detailed_instruct(task_description: str, query: str) -> str:
    return f"<instruct>{task_description}\n<query>{query}"


def get_detailed_example(task_description: str, query: str, response: str) -> str:
    return f"<instruct>{task_description}\n<query>{query}\n<response>{response}"


def get_new_queries(queries, query_max_len, examples_prefix, tokenizer):
    inputs = tokenizer(
        queries,
        max_length=query_max_len
        - len(tokenizer("<s>", add_special_tokens=False)["input_ids"])
        - len(tokenizer("\n<response></s>", add_special_tokens=False)["input_ids"]),
        return_token_type_ids=False,
        truncation=True,
        return_tensors=None,
        add_special_tokens=False,
    )
    prefix_ids = tokenizer(examples_prefix, add_special_tokens=False)["input_ids"]
    suffix_ids = tokenizer("\n<response>", add_special_tokens=False)["input_ids"]
    new_max_length = (
        len(prefix_ids) + len(suffix_ids) + query_max_len + 8
    ) // 8 * 8 + 8
    new_queries = tokenizer.batch_decode(inputs["input_ids"])
    for i in range(len(new_queries)):
        new_queries[i] = examples_prefix + new_queries[i] + "\n<response>"
    return new_max_length, new_queries


df_val_flat = df_val_flat.dropna(subset=["MisconceptionId"])
task = "Given a math multiple-choice problem with a student's wrong answer, retrieve the math misconceptions"
queries = [get_detailed_instruct(task, q) for q in df_val_flat["Prompt"]]
documents = df_mapping["MisconceptionName"].tolist()
query_max_len, doc_max_len = 320, 48
LORA_PATH = "../models/kaggle-models/2211-lora-14b-transformers-default-v1/"
tokenizer = AutoTokenizer.from_pretrained(LORA_PATH)
examples_prefix = ""
new_query_max_len, new_queries = get_new_queries(
    queries, query_max_len, examples_prefix, tokenizer
)
queries_embeddings = get_embeddings(new_queries)
documents_embeddings = get_embeddings(documents)


In [18]:
import torch
from typing import List


def compute_metrics(
    q_embeds: torch.Tensor, d_embeds: torch.Tensor, target_ids: List[int]
):
    """
    Compute MAP@25 and Recall@50 metrics.

    Args:
        q_embeds (torch.Tensor): Query embeddings of shape (M, dim), where M is the number of queries.
        d_embeds (torch.Tensor): Document embeddings of shape (N, dim), where N is the number of documents.
        target_ids (List[int]): List of target document indices (length M, one target index per query).

    Returns:
        None: Prints MAP@25 and Recall@100.
    """
    # Compute similarity scores
    scores = q_embeds @ d_embeds.T  # Shape: (M, N)

    # Initialize variables for metrics
    avg_precisions = []  # To store average precision for each query
    recall_counts = []  # To store recall@100 counts for each query

    # Compute metrics for each query
    for i, target_id in enumerate(target_ids):
        # Sort document indices by score in descending order
        sorted_indices = torch.argsort(scores[i], descending=True)

        relevant_docs = (sorted_indices[:500] == target_id).nonzero(as_tuple=True)[
            0
        ] 
        recall_count = (
            1 if len(relevant_docs) > 0 else 0
        )  # Check if target is in the top 100
        recall_counts.append(recall_count)

        # Compute average precision for top 25 (MAP@25)
        precision_at_k = 0.0
        num_relevant = 0
        for rank, idx in enumerate(sorted_indices[:25]):
            if idx == target_id:
                num_relevant += 1
                precision_at_k += num_relevant / (rank + 1)
        avg_precisions.append(precision_at_k / 1 if num_relevant > 0 else 0)

    # Calculate metrics
    map25 = sum(avg_precisions) / len(avg_precisions)
    recall50 = sum(recall_counts) / len(recall_counts)

    # Print results
    print(f"MAP@25: {map25:.4f}")
    print(f"Recall@50: {recall50:.4f}")

In [19]:
compute_metrics(
    torch.tensor(queries_embeddings),
    torch.tensor(documents_embeddings),
    df_val_flat["MisconceptionId"].tolist(),
)

MAP@25: 0.7561
Recall@50: 1.0000


# 4. Demo application

# 5. Model quantization
To further optimize the model for deployment, we quantize the model using AWQ. For best result, you should use an indomain dataset to calibrate the quantization.
```bash
!python scripts/convert_to_awq.py Qwen/Qwen2.5-0.5B \
    outputs/models/0.5B/round1/awq \
    --lora_path outputs/models/0.5B/round1/
```
```
# # Expected output:

# # Model already merged at outputs/models/0.5B/round1/awq/merged
# # Quantized model will be saved at outputs/models/0.5B/round1/awq/awq
# # Repo card metadata block was not found. Setting CardData to empty.
# # AWQ: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [03:13<00:00,  8.07s/it]
# # [2025-01-05 19:48:07,586] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)
# # Model is quantized and saved at "outputs/models/0.5B/round1/awq/awq"
# # ```